In [37]:
import warnings
import logging
import os
warnings.simplefilter('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.FATAL)

In [41]:
##IMPORTS##
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K
import os
import cv2
from glob import glob
#Imports for model
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.layers import Activation, add, multiply, Lambda
from tensorflow.keras.layers import AveragePooling2D, average, UpSampling2D, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.initializers import glorot_normal, random_normal, random_uniform
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

from tensorflow.keras import backend as K
from tensorflow.keras.layers import BatchNormalization 
from tensorflow.keras.applications import VGG19, densenet
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import Recall, Precision, MeanIoU

import matplotlib.pyplot as plt 
from sklearn.metrics import roc_curve, auc, precision_recall_curve # roc curve tools
from sklearn.model_selection import train_test_split

import time
import h5py

In [86]:
epsilon = 1e-5
smooth = 1

In [42]:

curdir = os.getcwd()
img_size = (128, 128)
num_classes_= 1
batch_size =32
train_input_paths = sorted(glob(os.path.join(curdir, 'train/x_train/*.jpg')))
train_target_paths = sorted(glob(os.path.join(curdir, 'train/y_train/*.png')))
valid_input_paths = sorted(glob(os.path.join(curdir, 'valid/x_valid/*.jpg')))
valid_target_paths = sorted(glob(os.path.join(curdir, 'valid/y_valid/*.png')))

In [66]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import Conv2DTranspose, concatenate

In [100]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img

class SkinLesionData(keras.utils.Sequence):
    def __init__(self, batch_size, img_size, input_paths, target_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_paths = input_paths
        self.target_paths = target_paths
        
    def __len__(self):
        return len(self.target_paths)//self.batch_size
    
    def __getitem__(self, idx):
        i = idx * self.batch_size
        batch_input_img_paths = self.input_paths[i:i+self.batch_size]
        batch_target_img_paths = self.target_paths[i:i+self.batch_size]
        x = np.zeros((self.batch_size, self.img_size[0], self.img_size[1] ,3), dtype="float32")
        
        for j, path in enumerate(batch_input_img_paths):
            img = plt.imread(path)
            img = cv2.resize(img, img_size)
            x[j] = img
         
        y = np.zeros((self.batch_size, self.img_size[0],self.img_size[1],1), dtype= "float32")
        
        for j, path in enumerate(batch_target_img_paths):
            img = plt.imread(path)
            img = cv2.resize(img , img_size)
            y[j] = np.expand_dims(img, 2)
        x = x/255
        return x,y
          

In [101]:
#Creating instance of the data loader
train_data = SkinLesionData(batch_size , img_size, train_input_paths, train_target_paths)
valid_data = SkinLesionData(batch_size, img_size, valid_input_paths, valid_input_paths)

In [46]:
def tversky(y_true, y_pred):

    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos))

    false_pos = K.sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.7
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)


In [47]:
def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true,y_pred)

def focal_tversky(y_true,y_pred):
    pt_1 = tversky(y_true, y_pred)
    gamma = 0.75
    return K.pow((1-pt_1), gamma)

In [48]:
#Dice Loss and Dice Score
def dsc(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dsc(y_true, y_pred)
    return loss

In [49]:

def tn(y_true, y_pred):
    smooth = 1
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos
    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos 

    tn = (K.sum(y_neg * y_pred_neg) + smooth) / (K.sum(y_neg) + smooth )
    return tn 

In [50]:
def tp(y_true, y_pred):
    smooth = 1
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pos = K.round(K.clip(y_true, 0, 1))
    tp = (K.sum(y_pos * y_pred_pos) + smooth)/ (K.sum(y_pos) + smooth) 
    return tp 

In [51]:
#IOU 
def iou(y_true, y_pred):
      def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + smooth) / (union + smooth)
        x = x.astype(np.float32)
        return x
      return tf.numpy_function(f, [y_true, y_pred], tf.float32)

In [56]:
def conv_block(inputs = None, num_filters = 32, dropout_prob=0, max_pooling = True):
  conv = Conv2D(num_filters, 3, 
                activation = 'relu', 
                padding = 'same', 
                kernel_initializer = 'he_normal')(inputs) 
  conv = Conv2D(num_filters, 3, 
                activation = 'relu', 
                padding = 'same', 
                kernel_initializer = 'he_normal')(conv)

  if dropout_prob > 0:
    conv = Dropout(dropout_prob)(conv)

  if max_pooling:
    next_layer = MaxPooling2D(pool_size=(2,2))(conv) 

  else:
    next_layer = conv

  skip_connection = conv

  return next_layer, skip_connection                 


In [57]:
def upsampling_block(expansive_input, contractive_input, n_filters = 32):
  up = Conv2DTranspose(n_filters,
                       3, 
                       strides = (2,2),
                       padding = 'same')(expansive_input)
  
  merge = concatenate([up, contractive_input], axis = 3)
  conv = Conv2D(n_filters, 
                3,
                padding = 'same',
                kernel_initializer = 'he_normal')(merge)
          
  conv = Conv2D(n_filters, 
                3,
                padding = 'same',
                kernel_initializer = 'he_normal')(conv)
  return conv

In [58]:
def unet_model(input_size = (128, 128, 3) , n_filter = 32 , n_classes = 1):
  inputs = Input(input_size)
  cblock1 = conv_block(inputs, n_filter) # 64*64
  cblock2 = conv_block(cblock1[0], n_filter*2) #32*32
  cblock3 = conv_block(cblock2[0], n_filter*4) # 16*16
  cblock4 = conv_block(cblock3[0], n_filter*8, dropout_prob=0.3) #8*8
  cblock5 = conv_block(cblock4[0], n_filter*16,dropout_prob=0.3, max_pooling=False)#8*8
  ublock6 = upsampling_block(cblock5[0], cblock4[1] ,n_filter*8 )
  ublock7 = upsampling_block(ublock6, cblock3[1] ,n_filter*4)
  ublock8 = upsampling_block(ublock7, cblock2[1] ,n_filter*2 )
  ublock9 = upsampling_block(ublock8, cblock1[1] ,n_filter )

  conv9 = Conv2D(n_filter, 3, activation = 'relu', padding ='same', kernel_initializer = 'he_normal')(ublock9)

  conv10  = Conv2D(n_classes, 1, padding ='same', activation = 'sigmoid')(conv9)

  model = tf.keras.Model(inputs = inputs, outputs = conv10)
  return model

In [59]:
model = unet_model()

In [60]:
model.compile(optimizer=Adam(learning_rate=1e-3), loss=focal_tversky, metrics=[iou,dsc,tp,tn])

In [ ]:
model.fit(train_data, validation_data=valid_data,epochs = 10)